# Introduction to geometry parsing using IfcOpenShell

This demo uses the IfcOpenShell [IfcOpenShell](https://ifcopenshell.org/), a powerful toolkit to interact with IFC files and follows some instructions of [https://thinkmoult.com/using-ifcopenshell-parse-ifc-files-python.html](https://thinkmoult.com/using-ifcopenshell-parse-ifc-files-python.html). 

## Installation
There are multiple ways to use IfcOpenShell and manage dependencies, see [IfcOpenShell_Installation](https://blenderbim.org/docs-python/ifcopenshell-python/installation.html). For using it with Anaconda, the `environment.yml` file is provided in this repository. Anyway, feel free to use whatever installation method suits you best. 





In [ ]:
import ifcopenshell

## Open an IFC file and explore its contents
We use an example file obtained from the Buildingsmart examples repository [https://github.com/buildingSMART/Sample-Test-Files](https://github.com/buildingSMART/Sample-Test-Files). The IFC 4.0 version is used as it is the recent stable release of IFC. The example might work with older versions like IFC 2x3 as well. Please note, that some specifications have changed so it might break. \
This screenshot gives you an idea of what the file contains visually: 
![IFC sample file](../data/ifc_sample_file_view.png)

In [ ]:
ifc_file = ifcopenshell.open("../data/wall-with-opening-and-window.ifc")

# what is in the ifc file
products = ifc_file.by_type('IfcProduct')
for product in products:
    print(product.is_a())

## Get the right element

As specified in the IFC standard, opening such as Windows and Doors always come with an IfcOpeningElement. Technically, the opening element causes the void in the parent element (wall, slab, ...). As its geometry is way less complex than the one of the window or door itself (of course depending on the level of detail), the opening is a good place to start to retrieve the geometric features of the opening for robotic task planning and other applications. 
The approach to filtering the correct element is pretty straightforward as the sample data only consists of a wall with a window and one opening element. So we just take the first element of the list of openings 

In [ ]:
# Get the opening element
opening = ifc_file.by_type("IfcOpeningElement")[0]
print("Return all information of opening as dicitonary:", opening.get_info())

## Local placement

What makes life a bit complex is that the stack of local placements needs to be considered. Local placements can be considered local coordinate systems. \
This figure illustrates how the placement interacts with the extruded area solid:
![Placement and Extruded Area Solid](../data/ifcextrudedareasolid-fig1.png)

In [ ]:
print("Check if there is a relative placement:",opening.ObjectPlacement.RelativePlacement)
print("Get the location of the relative placement:", opening.ObjectPlacement.RelativePlacement.Location)
relative_placement = tuple(opening.ObjectPlacement.RelativePlacement.Location[0])
print("Get the direction of the relative placement:", opening.ObjectPlacement.RelativePlacement.RefDirection)


## Geometric features

The geometry in IFC lives in the so called "Representation". As IFC is hierarchical and its complex can be a bit complex, some other classes appear. But technically, we need to get the Representation (again the first item of the list). Then we can exploit the various geometric information. \
In this example the Representation is an IfcExtrudedAreaSolid, a very common representation. 

In [ ]:
# Start digging into the geometry
# Geometry is in the IFC representation, so we retrieve the first item of the Representations list
geometry = opening.Representation.Representations[0].Items[0]
print("Representation of opening element:", geometry)

print("The placement (i. e. local coordinate system):", geometry.Position.Location[0]) 
print("Direction of the Extrusion:", geometry.ExtrudedDirection)
extrusion_depth = geometry.Depth 
print("Depth of Extrusion:", extrusion_depth) 
print("Extrusion area:", geometry.SweptArea) 
print("Poly line:", geometry.SweptArea.OuterCurve)
print("Points of the polyline round the extrusion area:", geometry.SweptArea.OuterCurve.Points) 


## Parse coordinates

We can now parse a list of coordinates of the extrusion area. You may notice that the coordinates are in the horizontal XY plane and must be transformed according to the local placement. The 8 corner points can be obtained by adding the Z component and the Extrusion depth

In [ ]:
points = geometry.SweptArea.OuterCurve.Points
print("Points from the IFC file:", points)

# in this simple example we will add the x, y and z components of the local placement to get the transformed coordinates
x_transform = relative_placement[0]
y_transform = relative_placement[1]
z_transform = relative_placement[2]

point_list = []
for point in points:
    for coord in point:
        # transform according to relative placement
        coord_transform = (coord[0] + x_transform, coord[1] + y_transform, z_transform)
        point_list.append(coord_transform)
        # add 4 more points with z + extrusion_depth as upper corners
        upper_corner = (coord[0] + x_transform, coord[1] + y_transform, z_transform + extrusion_depth)
        point_list.append(upper_corner)

print(point_list)

## Discussion

It is not mandatory for the opening element to be aligned with the vertical wall surfaces. In fact that is a point that is ignored in many cases. So, the coordinates of the opening do not necessarily give you the coordinates on the wall surface. Hence, checks and error handling need to be implemented to make this robust. 

[https://github.com/IfcOpenShell/IfcOpenShell/issues/551](https://github.com/IfcOpenShell/IfcOpenShell/issues/551)